In [41]:
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [3]:
# 读取输入数据
df_raw = pd.read_excel('华宝证券.xlsx', dtype={'成交日期': np.object, '证券代码': np.object, '印花税': np.float64, '过户费': np.float64, '成交费': np.float64, '委托编号': np.object})

In [9]:
# 读取分类表
df_category = pd.read_excel('资产配置分类表.xlsx',dtype={'基金代码': np.object, '分类ID': np.int64})
# df_category
# 基金名称	基金简称	基金代码	一级分类	二级分类	三级分类	分类ID	市场

In [44]:
# 按输出要求，调整数据表

def occur_money_calc(x):
#     print(x, type(x))
    if x['委托类别'] == '买入':
        return round(x['发生金额'] + x['fee'], 2)
    elif x['委托类别'] == '卖出':
        return round(x['发生金额'] - x['fee'], 2)
    pass

ignore_codes = ['799999']

# 调整输出
df_temp = df_raw[~df_raw['证券代码'].isin(ignore_codes)].copy()
df_temp = df_temp.reset_index()
df_temp['id'] = df_temp.index + 1
df_temp['date'] = df_temp['成交日期'].apply(lambda x: x[0:4]+'-'+x[4:6]+'-'+x[6:8])
df_temp['nav_acc'] = df_temp['成交价格']
df_temp['account'] = '华宝证券'
df_temp['note'] = '无'
df_temp['fee'] = df_temp['佣金'] + df_temp['印花税'] + df_temp['过户费'] + df_temp['成交费']
# '基金申购','托管转入' 都视为买入操作
df_temp = df_temp.replace(['基金申购','托管转入'], ['买入','买入'])
df_temp['occur_money'] = df_temp.apply(occur_money_calc, axis=1)
# 补充一二三级分类
df_temp = pd.merge(df_temp, df_category, left_on='证券代码', right_on='基金代码', how='left')
df_temp = df_temp.rename(columns={'成交时间':'time', '证券代码':'code', '证券名称':'name',
                                  '委托类别':'deal_type', '成交价格': 'nav_unit', '成交数量': 'volume',
                                  '发生金额': 'deal_money', '成本总计': 'fee', '一级分类': 'category1', 
                                  '二级分类': 'category2',  '三级分类': 'category3',  '分类ID': 'category_id', 
                                  '备注': 'note'})

df_temp = df_temp[['id','date','time','code','name','deal_type','nav_unit','nav_acc','volume','deal_money','fee','occur_money','account','category1','category2','category3','category_id','note']]
# 输出 csv 文件
df_temp.to_csv('huabao_output.csv')
# 输出 json 数据
json_list = json.loads(df_temp.to_json(orient='records', force_ascii=False))
[print(x) for x in json_list]

# TODO 看看是否有除了 “买入” 和 “卖出”之外的操作
# print(df_temp.deal_type.unique())

{'id': 1, 'date': '2020-02-03', 'time': '09:53:57', 'code': '162411', 'name': '华宝油气', 'deal_type': '买入', 'nav_unit': 0.36, 'nav_acc': 0.36, 'volume': 35500.0, 'deal_money': 12780.0, 'fee': 1.28, 'occur_money': 12781.28, 'account': '华宝证券', 'category1': '商品', 'category2': '商品', 'category3': '原油', 'category_id': 613, 'note': '无'}
{'id': 2, 'date': '2020-02-03', 'time': '11:26:14', 'code': '515180', 'name': '100红利', 'deal_type': '买入', 'nav_unit': 0.906, 'nav_acc': 0.906, 'volume': 2700.0, 'deal_money': 2446.2, 'fee': 0.24, 'occur_money': 2446.44, 'account': '华宝证券', 'category1': 'A 股', 'category2': '红利价值', 'category3': '中证红利', 'category_id': 131, 'note': '无'}
{'id': 3, 'date': '2020-02-10', 'time': '09:59:08', 'code': '515180', 'name': '100红利', 'deal_type': '卖出', 'nav_unit': 0.957, 'nav_acc': 0.957, 'volume': 2400.0, 'deal_money': 2296.8, 'fee': 0.23, 'occur_money': 2296.57, 'account': '华宝证券', 'category1': 'A 股', 'category2': '红利价值', 'category3': '中证红利', 'category_id': 131, 'note': '无'}
{'i

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]